# From Raw Video to Cropped Images of Badgers (using SpeciesNet)
This file takes the path for a raw video and creates both:
- a folder with the extracted frames (images) from the video (1 frame / second)
- a folder with the cropped images of badgers (using SpeciesNet to detect badgers in the original frames and then cropping using SpeciesNet bounding boxes).

## Input path to video file

In [4]:
# Directory of input images
INPUT_VID = r"C:\\Users\\md82\\OneDrive - Anglia Ruskin University\\Documents\\Python_code\\Badger-Cropping-Pipeline\\IMG_0011.AVI"  

# Names of output directories
extracted_frames_dir = "video_frames"
cropped_images_dir = "cropped_frames"

In [5]:
import os

def create_output_dirs(file_path, new_folder_name):
    # Get the directory containing the file
    parent_dir = os.path.dirname(file_path)

    # Create the full path for the new folder
    new_folder_path = os.path.join(parent_dir, new_folder_name)

    # Create the folder if it doesn't exist
    os.makedirs(new_folder_path, exist_ok=True)

## Create output directories

In [6]:
create_output_dirs(INPUT_VID, extracted_frames_dir)
create_output_dirs(INPUT_VID, cropped_images_dir)

## Extract frames from video

In [7]:
import os
import glob
from PIL import Image
from speciesnet.detector import SpeciesNetDetector
from Badger_detection import extract_frames
from tqdm import tqdm

c:\Users\md82\AppData\Local\anaconda3\envs\speciesnet\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
print("Step 1: Extracting frames...")
extract_frames(INPUT_VID, extracted_frames_dir, interval_seconds=1)

Step 1: Extracting frames...
Video properties: Total Frames=484, FPS=24.00, IntervalSec=1, Stride=24, Expected Saved Frames=21
No existing frames found in video_frames. Proceeding with extraction.
Extracted 21 frames from C:\\Users\\md82\\OneDrive - Anglia Ruskin University\\Documents\\Python_code\\Badger-Cropping-Pipeline\\IMG_0011.AVI to video_frames


(['video_frames\\frame_000000.jpg',
  'video_frames\\frame_000001.jpg',
  'video_frames\\frame_000002.jpg',
  'video_frames\\frame_000003.jpg',
  'video_frames\\frame_000004.jpg',
  'video_frames\\frame_000005.jpg',
  'video_frames\\frame_000006.jpg',
  'video_frames\\frame_000007.jpg',
  'video_frames\\frame_000008.jpg',
  'video_frames\\frame_000009.jpg',
  'video_frames\\frame_000010.jpg',
  'video_frames\\frame_000011.jpg',
  'video_frames\\frame_000012.jpg',
  'video_frames\\frame_000013.jpg',
  'video_frames\\frame_000014.jpg',
  'video_frames\\frame_000015.jpg',
  'video_frames\\frame_000016.jpg',
  'video_frames\\frame_000017.jpg',
  'video_frames\\frame_000018.jpg',
  'video_frames\\frame_000019.jpg',
  'video_frames\\frame_000020.jpg'],
 24.0003840061441)

## Detect badgers using SpeciesNet

In [9]:
# Name of video file
input_vid_name = os.path.basename(os.path.normpath(INPUT_VID))

# Threshold for SpeciesNet confidence
confidence_threshold = 0.6

# Get all image files in extracted frames folder
image_files = sorted(glob.glob(os.path.join(extracted_frames_dir, '*.jpg')) +
                    glob.glob(os.path.join(extracted_frames_dir, '*.jpeg')) +
                    glob.glob(os.path.join(extracted_frames_dir, '*.png')))
print(f"Found {len(image_files)} images in {extracted_frames_dir}")

# Initialize SpeciesNetDetector
print("Loading SpeciesNet Detector...")
detector = SpeciesNetDetector("kaggle:google/speciesnet/pyTorch/v4.0.1a")
print("✓ SpeciesNet Detector loaded.")

crop_count = 0
for idx, img_path in enumerate(tqdm(image_files, desc="Processing images")):
    image = Image.open(img_path).convert('RGB')
    preprocessed = detector.preprocess(image)
    result = detector.predict(img_path, preprocessed)
    detections = result.get('detections', [])
    for det_idx, det in enumerate(detections):
        label = det.get('label', '')
        conf = det.get('conf', 0)
        bbox = det.get('bbox', None)
        label_lower = label.lower()
        if (("meles" in label_lower) or ("badger" in label_lower) or ("animal" in label_lower)) and conf >= confidence_threshold and bbox is not None:
            # bbox is [x_min, y_min, width, height] in normalized coordinates
            x_min, y_min, width, height = bbox
            x0 = int(x_min * image.width)
            y0 = int(y_min * image.height)
            x1 = int((x_min + width) * image.width)
            y1 = int((y_min + height) * image.height)
            crop = image.crop((x0, y0, x1, y1))
            crop_filename = f"cropped_{input_vid_name}_frame{idx:06d}_det{det_idx}.jpg"
            crop_path = os.path.join(cropped_images_dir, crop_filename)
            crop.save(crop_path)
            crop_count += 1
print(f"Done! Saved {crop_count} cropped badger images to {cropped_images_dir}")


Found 21 images in video_frames
Loading SpeciesNet Detector...
✓ SpeciesNet Detector loaded.


Processing images: 100%|██████████| 21/21 [00:50<00:00,  2.43s/it]

Done! Saved 13 cropped badger images to cropped_frames
